# Time Series Analysis with TDA
First, import the usual libraries

In [1]:
import pandas as pd
import numpy as np
import gudhi as gd
from gudhi.point_cloud.timedelay import TimeDelayEmbedding
from gudhi.hera import wasserstein_distance
from gudhi.representations import PersistenceImage
from gudhi.representations.metrics import WassersteinDistance
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.manifold import MDS
import os
%matplotlib widget

Now we load the data.
We have five machines, for each of which we have time series for working well and for malfunctioning.
This way we get 10 numpy arrays in total.

In [2]:
datapath =".\\data\\timeseries\\good_new"
filelist = os.listdir(datapath)
print(sorted(filelist))
good_time_series = [np.array(pd.read_csv(os.path.join(datapath,f), header=None)[0]) for f in sorted(filelist)]
datapath =".\\data\\timeseries\\bad_new"
filelist = os.listdir(datapath)
print(sorted(filelist))
bad_time_series = [np.array(pd.read_csv(os.path.join(datapath,f), header=None)[0]) for f in sorted(filelist)]

print(good_time_series[0])

['sig1_good.txt', 'sig2_good.txt', 'sig3_good.txt', 'sig4_good.txt', 'sig5_good.txt']
['sig1_bad.txt', 'sig2_bad.txt', 'sig3_bad.txt', 'sig4_bad.txt', 'sig5_bad.txt']
[-0.11156466  0.01043311  0.30969176 ...  0.44117724  0.98839737
 -2.1306849 ]


Let us plot the time series

In [9]:
f,axes = plt.subplots(1,len(good_time_series), figsize = (25,5))
for i in range(0,len(good_time_series)):
    axes[i].plot(np.array(range(len(good_time_series[i]))),good_time_series[i], alpha = 1, c="tab:green")
    axes[i].plot(np.array(range(len(bad_time_series[i]))),bad_time_series[i], alpha=0.5, c="tab:red")
    #axes[i].plot(np.array(range(len(bad_time_series[i]))),bad_time_series[i]-good_time_series[i], alpha=0.5, c="tab:red")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now we use time delay embedding, mapping a time series valued in $\mathbb{R}$ to a point cloud in $\mathbb{R}^4$.
The parameters are determined by trial and error.
Concretely, using $dim =4, delay = 5, skip = 6$, we map the sequence
$$(x_1, x_2, x_3, x_4, x_5, x_6, \ldots)\mapsto ((x_1, x_6, x_{11}, x_{16}), (x_7,x_{12}, x_{17},x_{22}),(x_{13},x_{18},x_{23},x_{28})).$$

In [65]:
dim = 2
delay =716
skip = 1

tde = TimeDelayEmbedding(dim = dim, delay=delay, skip=skip)
good_point_clouds = tde.transform(good_time_series)
bad_point_clouds = tde.transform(bad_time_series)
diff_point_clouds = tde.transform(np.array(bad_time_series) - np.array(good_time_series))

print(np.array(good_point_clouds).shape)
print(good_point_clouds[0])

(5, 24284, 2)
[[-0.11156466  0.59295133]
 [ 0.01043311 -0.08505221]
 [ 0.30969176 -1.3842557 ]
 ...
 [ 0.03721146  0.44117724]
 [-0.28360267  0.98839737]
 [ 0.54943996 -2.1306849 ]]


In [66]:
f,ax = plt.subplots()
ax.scatter(diff_point_clouds[4][:,0],diff_point_clouds[4][:,1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

For each pointcloud, we compute persistent homology in dimension 1 using the alpha filtration.

In [67]:
good_pds = []

for pc in good_point_clouds:
    ac = gd.AlphaComplex(pc)
    st = ac.create_simplex_tree()
    st.compute_persistence()
    good_pds.append(st.persistence_intervals_in_dimension(1))

bad_pds = []

for pc in bad_point_clouds:
    ac = gd.AlphaComplex(pc)
    st = ac.create_simplex_tree()
    st.compute_persistence()
    bad_pds.append(st.persistence_intervals_in_dimension(1))

Now we plot the persistence diagrams.
The five pictures correspond to the five machines.
In each subfigure, the red diagram represents the well-working machine, whereas the green diagram represents the malfunctioning machine.

In [68]:
f,axes = plt.subplots(1,len(good_pds), figsize = (20,4))
for i in range(0,len(good_pds)):
    gd.plot_persistence_diagram(good_pds[i], axes=axes[i])
    gd.plot_persistence_diagram(bad_pds[i], axes=axes[i],colormap=cm.Set2.colors)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To do machine learning, we vectorize the diagrams to peristent images.
For illustration, we plot the PI corresponding to Machine 1 working well.

In [69]:
pi = PersistenceImage()
pi.fit(good_pds+bad_pds)
good_pis = pi.transform(good_pds)
bad_pis = pi.transform(bad_pds)
plt.matshow(good_pis[0].reshape(pi.resolution))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Next, we use MDS to visualize the data set in two dimensions.

In [70]:
pis = np.concatenate((good_pis,bad_pis))
mds = MDS(n_components=2)
emb =mds.fit_transform(pis)

In [71]:
f,ax = plt.subplots()
colors = ["tab:blue","tab:orange", "tab:green", "tab:red","tab:olive"]
ax.scatter(emb[:5,0],emb[:5,1], marker = "o", c= colors, label="well-working machine")
ax.scatter(emb[5:,0],emb[5:,1], marker = "x", c=colors, label="malfunctioning machine")
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
emb = pca.fit_transform(pis)

f,ax = plt.subplots()
colors = ["tab:blue","tab:orange", "tab:green", "tab:red","tab:olive"]
ax.scatter(emb[:5,0],emb[:5,1], marker = "o", c= colors, label="well-working machine")
ax.scatter(emb[5:,0],emb[5:,1], marker = "x", c=colors, label="malfunctioning machine")
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …